# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [2]:
import pandas as pd

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [21]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [22]:
json_normalize(data)

,counties,info.governor,shortname,state
0,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott,FL,Florida
1,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich,OH,Ohio


In [6]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [7]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,shortname,info.governor,state
0,Dade,12345,FL,Rick Scott,Florida
1,Broward,40000,FL,Rick Scott,Florida
2,Palm Beach,60000,FL,Rick Scott,Florida
3,Summit,1234,OH,John Kasich,Ohio
4,Cuyahoga,1337,OH,John Kasich,Ohio


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [9]:
# load json as string
data_list=json.load((open('data/world_bank_projects_less.json')))

In [10]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [119]:
# load json as string
data_list=json.load((open('data/world_bank_projects.json')))

In [137]:
type(data_list)

list

In [148]:
df=json_normalize(data_list,'mjtheme_namecode',['countrycode','countryname',['_id','$oid']],record_prefix='theme_')

In [149]:
df.head()

,theme_code,theme_name,countryname,countrycode,_id.$oid
0,8,Human development,Federal Democratic Republic of Ethiopia,ET,52b213b38594d8a2be17c780
1,11,,Federal Democratic Republic of Ethiopia,ET,52b213b38594d8a2be17c780
2,1,Economic management,Republic of Tunisia,TN,52b213b38594d8a2be17c781
3,6,Social protection and risk management,Republic of Tunisia,TN,52b213b38594d8a2be17c781
4,5,Trade and integration,Tuvalu,TV,52b213b38594d8a2be17c782


In [150]:
#Find the 10 countries with most projects, using project id as the unique identifier for each project

df.groupby('countryname')['_id.$oid'].nunique().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: _id.$oid, dtype: int64

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 5 columns):
theme_code     1499 non-null object
theme_name     1499 non-null object
countryname    1499 non-null object
countrycode    1499 non-null object
_id.$oid       1499 non-null object
dtypes: object(5)
memory usage: 58.6+ KB


In [152]:
#Find the top 10 major project themes (using column 'mjtheme_namecode')

df['theme_name'].value_counts()

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: theme_name, dtype: int64

In [198]:
#display all theme_code and theme_name pair so we know each theme_name's corresponding theme_code

df.groupby(['theme_code','theme_name'])[['theme_code','theme_name']].count()

theme_code  \
theme_code theme_name                                                 
1                                                                 5   
           Economic management                                   33   
10                                                               14   
           Rural development                                    202   
11                                                               27   
           Environment and natural resources management         223   
2                                                                15   
           Public sector governance                             184   
3                                                                 3   
           Rule of law                                           12   
4                                                                16   
           Financial and private sector development             130   
5                                                                 5   
           Trade and integration                                 72   
6                                                                10   
           Social protection and risk management                158   
7                                                                11   
           Social dev/gender/inclusion                          119   
8                                                                13   
           Human development                                    197   
9                                                                 3   
           Urban development                                     47   

                                                         theme_name  
theme_code theme_name                                                
1                                                                 5  
           Economic management                                   33  
10                                                               14  
           Rural development                                    202  
11                                                               27  
           Environment and natural resources management         223  
2                                                                15  
           Public sector governance                             184  
3                                                                 3  
           Rule of law                                           12  
4                                                                16  
           Financial and private sector development             130  
5                                                                 5  
           Trade and integration                                 72  
6                                                                10  
           Social protection and risk management                158  
7                                                                11  
           Social dev/gender/inclusion                          119  
8                                                                13  
           Human development                                    197  
9                                                                 3  
           Urban development                                     47

In [200]:
# replace null with corresponding theme_name if theme_name is ""

def replace_null (name,code):
    if name!="":
        return name
    elif code=='1':
        return 'Economic management'
    elif code=='2':
        return 'Public sector governance'
    elif code=='3':
        return 'Rule of law'
    elif code=='4':
        return 'Financial and private sector development'
    elif code=='5':
        return 'Trade and integration'
    elif code=='6':
        return 'Social protection and risk management'
    elif code=='7':
        return 'Social dev/gender/inclusion'
    elif code=='8':
        return 'Human development'
    elif code=='9':
        return 'Urban development'
    elif code=='10':
        return 'Rural development'
    else:
        return 'Environment and natural resources management'


In [202]:

df['theme_name']=df.apply(lambda row: replace_null (row['theme_name'],row['theme_code']),axis=1)


In [203]:
df.head()

,theme_code,theme_name,countryname,countrycode,_id.$oid
0,8,Human development,Federal Democratic Republic of Ethiopia,ET,52b213b38594d8a2be17c780
1,11,Environment and natural resources management,Federal Democratic Republic of Ethiopia,ET,52b213b38594d8a2be17c780
2,1,Economic management,Republic of Tunisia,TN,52b213b38594d8a2be17c781
3,6,Social protection and risk management,Republic of Tunisia,TN,52b213b38594d8a2be17c781
4,5,Trade and integration,Tuvalu,TV,52b213b38594d8a2be17c782


In [204]:
df.groupby(['theme_code','theme_name'])[['theme_code','theme_name']].count()

,,theme_code,theme_name
theme_code,theme_name,,
1,Economic management,38,38
10,Rural development,216,216
11,Environment and natural resources management,250,250
2,Public sector governance,199,199
3,Rule of law,15,15
4,Financial and private sector development,146,146
5,Trade and integration,77,77
6,Social protection and risk management,168,168
7,Social dev/gender/inclusion,130,130
